In [2]:
import pandas as pd
import numpy as np
import keras
import tensorflow
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense


data = pd.read_csv("train.txt", sep=';')
data.columns = ["Text", "Emotions"]
print(data.head())

                                                Text Emotions
0  i can go from feeling so hopeless to so damned...  sadness
1   im grabbing a minute to post i feel greedy wrong    anger
2  i am ever feeling nostalgic about the fireplac...     love
3                               i am feeling grouchy    anger
4  ive been feeling a little burdened lately wasn...  sadness


In [3]:
texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [4]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

In [5]:
# Encode the string labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [6]:
# One-hot encode the labels
one_hot_labels = keras.utils.to_categorical(labels)

In [7]:
# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences,
                                                one_hot_labels,
                                                test_size=0.2)

In [8]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1,
                    output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))

Epoch 1/10
400/400 [==============================] - 9s 22ms/step - loss: 1.3743 - accuracy: 0.4728 - val_loss: 0.9166 - val_accuracy: 0.7041
Epoch 2/10
400/400 [==============================] - 8s 21ms/step - loss: 0.3967 - accuracy: 0.8743 - val_loss: 0.5272 - val_accuracy: 0.8238
Epoch 3/10
400/400 [==============================] - 8s 20ms/step - loss: 0.0647 - accuracy: 0.9833 - val_loss: 0.5735 - val_accuracy: 0.8178
Epoch 4/10
400/400 [==============================] - 9s 22ms/step - loss: 0.0232 - accuracy: 0.9951 - val_loss: 0.5489 - val_accuracy: 0.8372
Epoch 5/10
400/400 [==============================] - 8s 19ms/step - loss: 0.0182 - accuracy: 0.9966 - val_loss: 0.5814 - val_accuracy: 0.8344
Epoch 6/10
400/400 [==============================] - 8s 21ms/step - loss: 0.0133 - accuracy: 0.9972 - val_loss: 0.6018 - val_accuracy: 0.8303
Epoch 7/10
400/400 [==============================] - 8s 20ms/step - loss: 0.0115 - accuracy: 0.9974 - val_loss: 0.6268 - val_accuracy: 0.8338

In [9]:
input_text = "i just feel really violent right now"

# Preprocess the input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label)

1/1 [==============================] - 0s 66ms/step
['anger']


In [10]:
#I might walk slow but never backwards
# I am feeling low
# I am hungry
#i was feeling an act of god at work in my life and it was an amazing feeling
#I am weeping
#i love dancing
#i think im just being stupid feeling nervous
#i just feel really violent right now